In [ ]:
import re
import csv
import sys
import json
import math
import spacy
import textacy
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from taxonerd import TaxoNERD
from fastcoref import spacy_component
from spacy.matcher import Matcher, DependencyMatcher, PhraseMatcher
%run "./Main.ipynb"

In [33]:
class Colors:
    HEADER = '\033[95m'
    RED = '\033[91m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [133]:
df = pd.read_csv("../Week 16/Datasets/Baseline-1.csv")

i = 0
title = df.loc[i, "Title"]
abstract = df.loc[i, "Abstract"]

nlp = spacy.load("en_core_web_trf")
doc = nlp(abstract)

# Title
print(f"{Colors.BOLD}{title}{Colors.ENDC}")

# Abstract
verbs = []
objects = []
subjects = []

for triple in textacy.extract.subject_verb_object_triples(doc):
    verbs.extend(triple.verb)
    objects.extend(triple.object)
    subjects.extend(triple.subject)
    
for token in doc:
    color = Colors.ENDC
    if token in objects:
        color = Colors.YELLOW
    if token in subjects:
        color = Colors.BLUE
    if token in verbs:
        color = Colors.RED

    if token.sent.end == token.i + 1:
        print(f"{color}{token.text} ", end=f"{Colors.ENDC}")
    elif token.nbor() and token.nbor().text in [".","?","!",";", ")", ",", "]"]:
        print(f"{color}{token.text}", end=f"{Colors.ENDC}")
    elif token.sent.start == token.i:
        print(f"{color}{token.text} ", end=f"{Colors.ENDC}")
    elif token.text not in ["(", "["]:
        print(f"{color}{token.text} ", end=f"{Colors.ENDC}")
    else:
        print(f"{color}{token.text}", end=f"{Colors.ENDC}")

Moving beyond linear food chains: trait-mediated indirect interactions in a rocky intertidal food web
In simple, linear food chains, top predators can have positive indirect effects on basal resources by causing changes in the traits (e.g. behaviour, feeding rates) of intermediate consumers. Although less is known about trait - mediated indirect interactions (TMIIs) in more complex food webs, it has been suggested that such complexity dampens trophic cascades. We examined TMIIs between a predatory crab (Carcinus maenas) and two ecologically important basal resources, fucoid algae (Ascophyllum nodosum) and barnacles (Semibalanus balanoides), which are consumed by herbivorous (Littorina littorea) and carnivorous (Nucella lapillus) snails, respectively. Because crab predation risk suppresses snail feeding rates, we hypothesized that crabs would also shape direct and indirect interactions among the multiple consumers and resources. We found that the magnitude of TMIIs between the crab and 

In [12]:
main = Main()
main.update_text(abstract)

C:\Users\lbeln\anaconda3\envs\3.10\lib\site-packages\thinc\shims\pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map

In [54]:
class Node:
    def __init__(self, main, is_action=False):
        self.main = main
        self.tokens = []
        self.expanded_tokens = []
        self.is_action = is_action

    def traits(self):
        tokens = set([*self.tokens, *self.expanded_tokens])
        tokens = tokens & set(self.main.trait.tokens)
        return tokens

    def species(self):
        tokens = set([*self.tokens, *self.expanded_tokens])
        tokens = tokens & set(self.main.species.tokens)
        return tokens

    def start(self):
        i = math.inf
        for token in self.tokens:
            i = min(i, token.i)
        return i

    def end(self):
        i = -math.inf
        for token in self.tokens:
            i = max(i, token.i)
        return i

    def __str__(self):
        tokens = list(set([*self.tokens, *self.expanded_tokens]))
        tokens = sorted(tokens, key=lambda token: token.i)
        return f"{','.join([t.text for t in tokens])}"

class Order:
    def __init__(self, main):
        self.main = main
        self.order = []

    def start(self):
        i = math.inf
        for item in self.order:
            i = min(i, item.start())
        return i

    def end(self):
        i = -math.inf
        for item in self.order:
            i = max(i, item.start())
        return i

    def __str__(self):
        ret = ""

        i = 0
        size = len(self.order)
        while i < size:
            ret += f"({self.order[i]})"
            if i != size - 1:
                ret += "->"
            i += 1
        
        return ret

In [131]:
def distinct_bounds(bounds):
    max_coverage = []

    for bound in bounds:
        overlap = False
        for i, max_bound in enumerate(max_coverage):
            contains = max_bound[0] <= bound[0] <= max_bound[1] or max_bound[0] <= bound[1] <= max_bound[1]
            surround = bound[0] <= max_bound[0] <= bound[1] or bound[0] <= max_bound[1] <= bound[1]
            
            if contains or surround:
                overlap = True
            
                if bound[1] - bound[0] > max_bound[1] - max_bound[0]:
                    max_coverage[i] = bound
        
        if not overlap:
            max_coverage.append(bound)

    return max_coverage

def expand_token(main, sent_i, token):
    expanded_token = [token]
    
    if token.pos_ == "PRON":
        expanded_token = [*main.coref_map.get(token, [token])]

    i = 0
    size = len(expanded_token)
    while i < size:
        exp_token = expanded_token[i]
        for ent_pos, ent in main.parts.reg[sent_i].items():
            if ent_pos[0] <= exp_token.i <= ent_pos[1] and ent.label in [Entity.ITEM]:
                expanded_token = [*main.sp_doc[ent_pos[0]:ent_pos[1]+1]]
                break
        i += 1

    i = 0
    size = len(expanded_token)
    while i < size:
        exp_token = expanded_token[i]
        if exp_token in main.noun_chunk_map:
            expanded_token.extend([*main.noun_chunk_map[exp_token]])

        if exp_token in main.ent_map:
            expanded_token.extend([*main.ent_map[exp_token]])

        i += 1
    
    expanded_token = list(set(expanded_token))
    
    return expanded_token

def order_nodes(s_node, v_node, o_node):
    # Subject Tokens Transferred to Object
    sub_transfer_tokens = []
    for token in s_node.tokens:
        if token.pos_ == "VERB":
            sub_transfer_tokens.append(token)
    
    s_node.tokens = [tkn for tkn in s_node.tokens if tkn not in sub_transfer_tokens]
    o_node.tokens.extend(sub_transfer_tokens)
    
    # Object Tokens Transferred to Verb
    obj_transfer_tokens = []
    for token in o_node.tokens:
        if token in main.cause.tokens or token in main.change.tokens:
            obj_transfer_tokens.append(token)
    
    o_node.tokens = [tkn for tkn in o_node.tokens if tkn not in obj_transfer_tokens]
    v_node.tokens.extend(obj_transfer_tokens)

    # Expand Tokens in Subject and Object
    s_expanded_tokens = flatten([expand_token(main, sent_i, tkn) for tkn in s_node.tokens])
    s_node.expanded_tokens = s_expanded_tokens
    
    o_expanded_tokens = flatten([expand_token(main, sent_i, tkn) for tkn in o_node.tokens])
    o_node.expanded_tokens = o_expanded_tokens

    # Create Order S -> V -> 0
    if (
        not s_node.tokens or 
        not v_node.tokens or 
        not o_node.tokens
    ):
        return None
    
    order = Order(main)
    order.order = [s_node, v_node, o_node]
    return order

def order_tokens(main, sent_i, tokens):
    if len(tokens) <= 2:
        return None
    
    # Sort Tokens by Position in Doc
    tokens = sorted(tokens, key=lambda token: token.i)
    
    verbs = [token for token in tokens if token.pos_ == "VERB"]
    if not verbs:
        return

    # Sort Verbs by Token Position in Doc
    verbs = sorted(verbs, key=lambda token: token.i)
    verb = verbs[0]

    if verb == tokens[0]:
        return None

    # Partition Tokens in Doc (L)
    l = tokens[0].i

    # Partition Tokens in Doc (R)
    i = tokens.index(verb) + 1
    while i < len(tokens) and tokens[i].pos_ not in ["PROPN", "NOUN", "PRON"]:
        i += 1
    
    if i <= 0 or i >= len(tokens):
        return None
    
    r = tokens[i].i

    # Swap Subj. and Obj.
    aux = verb.nbor(-1) and verb.nbor(-1).pos_ == "AUX"
    adp = verb.nbor(1) and verb.nbor(1).lower_ == "by"
    swap = aux or adp

    # Creating Nodes
    v_node = Node(main, is_action=True)
    v_node.tokens = [verb]

    a_node = Node(main)
    a_node.tokens = [main.sp_doc[i] for i in range(l, verb.i)]
    if not a_node.tokens:
        return None

    b_node = Node(main)
    b_node.tokens = [main.sp_doc[i] for i in range(verb.i+1, r+1)]
    if not b_node.tokens:
        return None

    if swap:
        s_node = b_node
        o_node = a_node
    else:
        s_node = a_node
        o_node = b_node

    order = order_nodes(s_node, v_node, o_node)
    return order

def order_entity(main, sent_i, ent):
    assert ent.doc == main.sp_doc
    ent_tokens = [main.sp_doc[i] for i in range(ent.l, ent.r+1)]
    return order_tokens(main, sent_i, ent_tokens)

def order_document(main):
    sents = list(main.sp_doc.sents)
    sents_orders = {sent.start: [] for sent in sents}
    sents_triples = {sent.start: [] for sent in sents}

    # Subject-Verb-Object Triples
    for triple in textacy.extract.subject_verb_object_triples(doc):
        sents_triples[triple.verb[0].sent.start].append(triple)

    for sent_i, sent in enumerate(sents):
        # Sentence SVOs
        for triple in sents_triples[sent.start]:
            print(f"{Colors.BOLD}{Colors.BLUE}{triple}{Colors.ENDC}")

            # Create Verb Node
            v_node = Node(main, is_action=True)
            v_node.tokens = [*triple.verb]

            # Find Subject, Object Nodes
            # We don't know exactly which is which.
            # However, we assume that the verb is
            # between the subject and object.
            svo_tokens = [*triple.subject, *triple.object]
            svo_tokens = sorted(svo_tokens, key=lambda token: token.i)
            
            l = svo_tokens[0].i
            r = svo_tokens[-1].i

            # L and R Positions of Verb Tokens
            l_m = triple.verb[0].i
            r_m = triple.verb[-1].i
            
            a_node = Node(main)
            a_node.tokens = [main.sp_doc[i] for i in range(l, l_m)]
            a_node.expanded_tokens =  flatten([expand_token(main, sent_i, tkn) for tkn in a_node.tokens])
            
            b_node = Node(main)
            b_node.tokens = [main.sp_doc[i] for i in range(r_m+1, r+1)]
            b_node.expanded_tokens =  flatten([expand_token(main, sent_i, tkn) for tkn in b_node.tokens])

            # Check to Switch Subject and Object
            start_verb = triple.verb[0]
            aux = start_verb.nbor(-1) and start_verb.nbor(-1).pos_ == "AUX"
            adp = start_verb.nbor(1) and start_verb.nbor(1).lower_ == "by"
            swap = aux or adp
            
            if swap:
                s_node = b_node
                o_node = a_node
            else:
                s_node = a_node
                o_node = b_node

            order = order_nodes(s_node, v_node, o_node)
            sents_orders[sent.start].append(order)

            # Find (Nested) Relationships (Orders) in
            # Subject and Object
            sub_order = order_tokens(main, sent_i, s_node.tokens)   
            obj_order = order_tokens(main, sent_i, o_node.tokens)
            sents_orders[sent.start].extend([sub_order, obj_order])

        # Find Relationships (Orders) in Entities
        ent_bounds = list(main.parts.reg[sent_i].keys())
        distinct_ent_bounds = distinct_bounds(ent_bounds)
        entities = [ent[1] for ent in main.parts.reg[sent_i].items() if ent[0] in distinct_ent_bounds]
        
        for entity in entities:
            ent_order = order_entity(main, sent_i, entity)
            sents_orders[sent.start].append(ent_order)

    return sents_orders

In [132]:
ret = order_document(main)

SVOTriple(subject=[predators], verb=[can, have], object=[effects])
SVOTriple(subject=[complexity], verb=[dampens], object=[cascades])
SVOTriple(subject=[We], verb=[examined], object=[TMIIs])
SVOTriple(subject=[which], verb=[are, consumed], object=[Littorina, littorea, carnivorous, snails])
SVOTriple(subject=[predation, risk], verb=[suppresses], object=[feeding, rates])
SVOTriple(subject=[crabs], verb=[would, shape], object=[interactions])
SVOTriple(subject=[snails], verb=[transmitted], object=[TMIIs])
SVOTriple(subject=[algae, TMIIs], verb=[were, transmitted], object=[snails])
SVOTriple(subject=[Nucella], verb=[to, consume], object=[barnacles])
SVOTriple(subject=[predation, risk], verb=[shapes], object=[community, dynamics])
